## Generate Hyp3 interferogram stack

This notebook will queue up processing jobs in Hyp3 and then download and prepare them for use in Mintpy. Code adapted from [Hyp3 tutorial](https://nbviewer.org/github/ASFHyP3/hyp3-docs/blob/main/docs/tutorials/hyp3_insar_stack_for_ts_analysis.ipynb).


In [1]:
site = 'NorthSlopeEastD102'
year = 2023

### Prepare environment

Import functions and load metadata for processing. The environment requires several packages from the Alaska Satellite Facility to function, specifically [asf_search](https://docs.asf.alaska.edu/asf_search/basics/) and [hyp3_sdk](https://hyp3-docs.asf.alaska.edu/using/sdk/).

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
import os
from dateutil.parser import parse as parse_date
from typing import List, Union
from osgeo import gdal
from solid_utils import permafrost_utils as pu
import asf_search as asf
import hyp3_sdk as sdk

In [3]:
################# Set Directories ##########################################
print('\nCurrent directory:',os.getcwd())

if 'work_dir' not in locals():
    work_dir = Path.cwd()/'work'/'permafrost_ouputs'/site/str(year)

print("Work directory:", work_dir)
work_dir.mkdir(parents=True, exist_ok=True)
# Change to Workdir   
os.chdir(work_dir)
       
hyp3_dir = work_dir/'products'               #aka gunwdir
hyp3_dir.mkdir(parents=True, exist_ok=True)
print("   Hyp3  dir:", hyp3_dir) 
    
mintpy_dir = work_dir/'MintPy' 
mintpy_dir.mkdir(parents=True, exist_ok=True)
print("   MintPy  dir:", mintpy_dir)
############################################################################
### List of CalVal Sites:
'''
Set NISAR calval sites:
    NorthSlopeEastD102      : North Slope of Alaska including Dalton highway, Sentinel-1 descending path (track) 102. 
                            : The field validation sites are located in this frame.
    NorthSlopeWestD44       : Western area of the North Slope of Alaska, Sentinel-1 descending path (track) 44.
    NorthwestTerritoriesA78 : North east of Yellowknife, Sentinel-1 ascending path (track) 78.


Hyp3 & MintPy parameters:
    calval_location    : name
    region_identifier  : WTK string with latlon point for identifying image in Hyp3
    subset_region      : subset analysis area, in UTM. Given in '[ymin:ymax,xmin:xmax]' or 'none'
    download_start_date : download start date as YYYMMDD  
    download_end_date   : download end date as YYYMMDD
    mintpy_ref_loc     : reference point for use in mintpy in UTM Y,X order. Projection must be that of the Hyp3 images.
    tempBaseMax        : maximum number of days, don't use interferograms longer than this value 
    ifgExcludeList     : default is not to exclude any interferograms
    maskWater          : interior locations don't need to mask water
    sentinel_path      : asfPath number for identifying image. Also called track.
    sentinel_frame     : asfFrame number for identifying image
'''
sites = {
    ##########  NORTH SLOPE EAST (DALTON/TOOLIK) ##############
    'NorthSlopeEastD102' : {'calval_location' : 'NorthSlopeEastD102',
            'region_identifier' : 'POINT(-149.37 69.09)',
            'subset_region' : '[7620213:7686754, 641941:679925]',            
            'download_start_date' : '20230525',
            'download_end_date'   : '20230910',
            'mintpy_ref_loc'      : '7651392, 666923',
            'tempBaseMax' : '36',
            'ifgExcludeList' : 'auto',
            'maskWater' : 'True',
            'sentinel_direction' : 'DESCENDING',
            'sentinel_path' : '102',
            'sentinel_frame' : '362'},
    'NorthSlopeWestD44' : {'calval_location' : 'NorthSlopeWestD44',
            'region_identifier' : 'POINT(-160.25 68.82)',
            'subset_region' : 'none',
            'download_start_date' : '20230525',
            'download_end_date'   : '20230910',
            'mintpy_ref_loc' : '7684081, 437118',
            'tempBaseMax' : '36',
            'ifgExcludeList' : 'auto',
            'maskWater' : 'True',
            'sentinel_direction' : 'DESCENDING',
            'sentinel_path' : '44',
            'sentinel_frame' : '362'},
    'NorthwestTerritoriesA78' : {'calval_location' : 'NorthwestTerritoriesA78',
            'region_identifier' : 'POINT(-111.35 63.20)',
            'subset_region' : 'none',
            'download_start_date' : '20230525',
            'download_end_date'   : '20230910',
            'mintpy_ref_loc'      : '6980804, 522414',
            'tempBaseMax' : '36',
            'ifgExcludeList' : 'auto',
            'maskWater' : 'True',
            'sentinel_direction' : 'ASCENDING',
            'sentinel_path' : '78',
            'sentinel_frame' : '204'},    
}


Current directory: /home/jovyan/NISAR_cal
Work directory: /home/jovyan/NISAR_cal/work/permafrost_ouputs/NorthSlopeEastD102/2023
   Hyp3  dir: /home/jovyan/NISAR_cal/work/permafrost_ouputs/NorthSlopeEastD102/2023/products
   MintPy  dir: /home/jovyan/NISAR_cal/work/permafrost_ouputs/NorthSlopeEastD102/2023/MintPy


### Generate SBAS image pairs

Identify stack of SLC images and determine SBAS pairs.

In [6]:
lonlat = sites[site]['download_region']
frame = sites[site]['sentinel_frame']
if sites[site]['sentinel_direction']=='ASCENDING':
    direction = asf.ASCENDING
elif sites[site]['sentinel_direction']=='DESCENDING':
    direction = asf.DESCENDING

stack_start = parse_date(f'{year}-05-25 00:00:00Z')
stack_end = parse_date(f'{year}-09-10 00:00:00Z')

search_results = asf.geo_search(platform=asf.SENTINEL1, intersectsWith=lonlat, start=stack_start,
    end=stack_end, processingLevel=asf.SLC,beamMode=asf.IW,flightDirection=direction,asfFrame=frame)

In [7]:
baseline_results = asf.baseline_search.stack_from_product(search_results[-1])

columns = list(baseline_results[0].properties.keys()) + ['geometry', ]
data = [list(scene.properties.values()) + [scene.geometry, ] for scene in baseline_results]

stack = pd.DataFrame(data, columns=columns)
stack['startTime'] = stack.startTime.apply(parse_date)

# stack_start = parse_date(f'{year}-05-25 00:00:00Z')
# stack_end = parse_date(f'{year}-09-10 00:00:00Z')

stack = stack.loc[(stack_start <= stack.startTime) & (stack.startTime <= stack_end)]

stack

9


,beamModeType,browse,bytes,centerLat,centerLon,faradayRotation,fileID,flightDirection,groupID,granuleType,...,sceneName,sensor,startTime,stopTime,url,fileName,frameNumber,temporalBaseline,perpendicularBaseline,geometry
182,IW,None,4608855607,68.9321,-151.7568,None,S1A_IW_SLC__1SDV_20230603T164401_20230603T1644...,DESCENDING,S1A_IWDV_0361_0368_048824_102,SENTINEL_1A_FRAME,...,S1A_IW_SLC__1SDV_20230603T164401_20230603T1644...,C-SAR,2023-06-03 16:44:01+00:00,2023-06-03T16:44:29.000Z,https://datapool.asf.alaska.edu/SLC/SA/S1A_IW_...,S1A_IW_SLC__1SDV_20230603T164401_20230603T1644...,362,0,0.0,"{'coordinates': [[[-149.293701, 67.836151], [-..."
183,IW,None,4686505900,68.9316,-151.7565,None,S1A_IW_SLC__1SDV_20230615T164402_20230615T1644...,DESCENDING,S1A_IWDV_0361_0368_048999_102,SENTINEL_1A_FRAME,...,S1A_IW_SLC__1SDV_20230615T164402_20230615T1644...,C-SAR,2023-06-15 16:44:02.311000+00:00,2023-06-15T16:44:30.221Z,https://datapool.asf.alaska.edu/SLC/SA/S1A_IW_...,S1A_IW_SLC__1SDV_20230615T164402_20230615T1644...,362,12,-16.0,"{'coordinates': [[[-149.29332, 67.835739], [-1..."
184,IW,None,4688870133,68.9313,-151.7526,None,S1A_IW_SLC__1SDV_20230627T164402_20230627T1644...,DESCENDING,S1A_IWDV_0361_0368_049174_102,SENTINEL_1A_FRAME,...,S1A_IW_SLC__1SDV_20230627T164402_20230627T1644...,C-SAR,2023-06-27 16:44:02.446000+00:00,2023-06-27T16:44:30.361Z,https://datapool.asf.alaska.edu/SLC/SA/S1A_IW_...,S1A_IW_SLC__1SDV_20230627T164402_20230627T1644...,362,24,-172.0,"{'coordinates': [[[-149.289795, 67.835335], [-..."
185,IW,None,4678720995,68.9313,-151.7543,None,S1A_IW_SLC__1SDV_20230709T164403_20230709T1644...,DESCENDING,S1A_IWDV_0361_0368_049349_102,SENTINEL_1A_FRAME,...,S1A_IW_SLC__1SDV_20230709T164403_20230709T1644...,C-SAR,2023-07-09 16:44:03.895000+00:00,2023-07-09T16:44:31.808Z,https://datapool.asf.alaska.edu/SLC/SA/S1A_IW_...,S1A_IW_SLC__1SDV_20230709T164403_20230709T1644...,362,36,-107.0,"{'coordinates': [[[-149.291443, 67.835373], [-..."
186,IW,None,4689162092,68.9314,-151.7562,None,S1A_IW_SLC__1SDV_20230721T164404_20230721T1644...,DESCENDING,S1A_IWDV_0361_0368_049524_102,SENTINEL_1A_FRAME,...,S1A_IW_SLC__1SDV_20230721T164404_20230721T1644...,C-SAR,2023-07-21 16:44:04.486000+00:00,2023-07-21T16:44:32.398Z,https://datapool.asf.alaska.edu/SLC/SA/S1A_IW_...,S1A_IW_SLC__1SDV_20230721T164404_20230721T1644...,362,48,-23.0,"{'coordinates': [[[-149.293121, 67.835464], [-..."
187,IW,None,4678115605,68.9315,-151.7550,None,S1A_IW_SLC__1SDV_20230802T164404_20230802T1644...,DESCENDING,S1A_IWDV_0361_0368_049699_102,SENTINEL_1A_FRAME,...,S1A_IW_SLC__1SDV_20230802T164404_20230802T1644...,C-SAR,2023-08-02 16:44:04.784000+00:00,2023-08-02T16:44:32.697Z,https://datapool.asf.alaska.edu/SLC/SA/S1A_IW_...,S1A_IW_SLC__1SDV_20230802T164404_20230802T1644...,362,60,-107.0,"{'coordinates': [[[-149.29213, 67.83564], [-14..."
188,IW,None,4614840459,68.9313,-151.7536,None,S1A_IW_SLC__1SDV_20230814T164405_20230814T1644...,DESCENDING,S1A_IWDV_0361_0368_049874_102,SENTINEL_1A_FRAME,...,S1A_IW_SLC__1SDV_20230814T164405_20230814T1644...,C-SAR,2023-08-14 16:44:05.666000+00:00,2023-08-14T16:44:33.581Z,https://datapool.asf.alaska.edu/SLC/SA/S1A_IW_...,S1A_IW_SLC__1SDV_20230814T164405_20230814T1644...,362,72,-110.0,"{'coordinates': [[[-149.290665, 67.835297], [-..."
189,IW,None,4671684100,68.9310,-151.7538,None,S1A_IW_SLC__1SDV_20230826T164406_20230826T1644...,DESCENDING,S1A_IWDV_0361_0368_050049_102,SENTINEL_1A_FRAME,...,S1A_IW_SLC__1SDV_20230826T164406_20230826T1644...,C-SAR,2023-08-26 16:44:06.579000+00:00,2023-08-26T16:44:34.495Z,https://datapool.asf.alaska.edu/SLC/SA/S1A_IW_...,S1A_IW_SLC__1SDV_20230826T164406_20230826T1644...,362,84,-118.0,"{'coordinates': [[[-149.291, 67.83493], [-148...."
190,IW,None,4628564907,68.9322,-151.7518,None,S1A_IW_SLC__1SDV_20230907T164407_20230907T1644...,DESCENDING,S1A_IWDV_0361_0368_050224_102,SENTINEL_1A_FRAME,...,S1A_IW_SLC__1SDV_20230907T164407_20230907T1644...,C-SAR,2023-09-07 16:44:07.029000+00:00,2023-09-07T16:44:34.946Z,https://d

In [8]:
sbas_pairs = set()

for reference, rt in stack.loc[::-1, ['sceneName', 'temporalBaseline']].itertuples(index=False):
    secondaries = stack.loc[
        (stack.sceneName != reference)
        & (stack.temporalBaseline - rt <= int(sites[site]['tempBaseMax']))
        & (stack.temporalBaseline - rt > 0)
    ]
    for secondary in secondaries.sceneName:
        sbas_pairs.add((reference, secondary))

### Do Hyp3 Processing

This step requires NASA Earthdata credentials. See [Hyp3 documentation](https://hyp3-docs.asf.alaska.edu/) for more information about using Hyp3 processing. Each user has an allotted Hyp3 processing quota, for which some of this processing is counted against.

In [ ]:

hyp3 = sdk.HyP3(prompt=True)


In [8]:

jobs = sdk.Batch()
h3projname = f"{site}_NCV_{year}"
for reference, secondary in sbas_pairs:
    jobs += hyp3.submit_insar_job(reference, secondary, name=h3projname,
                                  include_dem=True, include_look_vectors=True)


Jobs were set to be processed by this command!


This processing will likely take about an hour to complete. You cannot download data until the processing is finished. The following cell will check to see if the processing is still running.

In [8]:
class stillrunningError(Error):
    """raised when jobs are still running"""
    pass

running = len(hyp3.find_jobs(status_code=["RUNNING"]))
running += len(hyp3.find_jobs(status_code=["PENDING"]))

if running > 0:
    print('Jobs still running!')
    raise stillrunningError
else:
    print('No jobs running, all clear')

No jobs running, all clear


### Download Data

In [ ]:

h3projname = f"{site}_NCV_{year}"
batch = hyp3.find_jobs(name=h3projname)
insar_products = batch.download_files(hyp3_dir)
insar_products = [sdk.util.extract_zipped_product(ii) for ii in insar_products]


### Crop Interferograms

In [11]:
# Crop to common overlap
files = hyp3_dir.glob('*/*_dem.tif')
overlap = pu.get_common_overlap(files)
pu.clip_hyp3_products_to_common_overlap(hyp3_dir, overlap)